# OECD HAN Database

In [4]:
# load packages 
library(pacman)
p_load(tidyverse, data.table, dtplyr, reshape2, archive, kableExtra)
# set option
options(dplyr.summarise.inform = FALSE)
gray_scale <- c('#F3F4F8','#D2D4DA', '#B3B5BD', 
                '#9496A1', '#7d7f89', '#777986', 
                '#656673', '#5B5D6B', '#4d505e',
                '#404352', '#2b2d3b', '#282A3A',
                '#1b1c2a', '#191a2b',
                '#141626', '#101223')

In [6]:
han_names <- fread('work/notebooks/patent/data/202208_HAN_NAMES.txt')

In [7]:
dim(han_names)

[1] 4191007       3

In [7]:
head(han_names)

HAN_ID,Clean_name,Person_ctry_code
<int>,<chr>,<chr>
1,& HAMBOURG NIENDORF,DE
2,& KK,JP
3,“ASTRONIT” CLOSE CORP,RU
4,“DEUTSCHE SEE” GMBH,DE
5,“EFIRNOIE” OPEN JOINT STOCK CO,RU
6,“EUROSTANDART” LTD LIABILITY CO,RU


In [8]:
de_firms <- fread('work/notebooks/patent/data/orbis_de_matched_l.csv')

In [55]:
han_names %>%
    .[1:5,] %>%
    kable('pipe', align='ccc')



| HAN_ID |           Clean_name           | Person_ctry_code |
|:------:|:------------------------------:|:----------------:|
|   1    |      & HAMBOURG NIENDORF       |        DE        |
|   2    |              & KK              |        JP        |
|   3    |     “ASTRONIT” CLOSE CORP      |        RU        |
|   4    |      “DEUTSCHE SEE” GMBH       |        DE        |
|   5    | “EFIRNOIE” OPEN JOINT STOCK CO |        RU        |

In [53]:
de_firms %>%
    .[1:5, 2:3]

name_native,name_internat
<chr>,<chr>
Airbus Defence and Space GmbH,Airbus Defence and Space GmbH
EurA AG,EurA AG
TuTech Innovation GmbH,TuTech Innovation GmbH
FFT Produktionssysteme GmbH & Co. KG.,FFT Produktionssysteme GmbH & Co. KG.
Diehl Aviation Laupheim GmbH,Diehl Aviation Laupheim GmbH


In [15]:
# query
airbus <- toupper('Airbus Defence')
han_names %>%
    .[Person_ctry_code == 'DE'] %>%
    .[Clean_name %like% airbus] %>%
    .[,HAN_ID] -> airbus_han_ids
airbus_han_ids

[1]   60513   62422 3637004 4401227 4527012

In [20]:
han_person <- fread('work/notebooks/patent/data/202208_HAN_PERSON.txt')

In [22]:
han_person %>%
    .[HAN_ID %in% airbus_han_ids]

HAN_ID,HARM_ID,Person_id,Person_name_clean,Person_ctry_code,Matched
<int>,<int>,<int>,<chr>,<chr>,<int>
60513,60513,53436663,AIRBUS DEFENCE & SPACE GMBH,DE,1
60513,60513,54048509,AIRBUS DEFENCE & SPACE GMBH,DE,1
60513,60513,72444595,AIRBUS DEFENCE & SPACE GMBH,DE,1
60513,60513,51178870,AIRBUS DEFENCE & SPACE GMBH,DE,1
60513,60513,53550520,AIRBUS DEFENCE & SPACE GMBH,DE,1
60513,60513,55417987,AIRBUS DEFENCE & SPACE GMBH,DE,1
60513,60513,47862439,AIRBUS DEFENCE & SPACE GMBH,DE,1
60513,60513,51379857,AIRBUS DEFENCE & SPACE GMBH,DE,1
60513,60513,52065724,AIRBUS DEFENCE & SPACE GMBH,DE,1


In [23]:
han_patents <- fread('work/notebooks/patent/data/202208_HAN_PATENTS.txt')

In [31]:
han_patents %>%
    .[HAN_ID %in% airbus_han_ids] %>%
    .[sample(.N, 5)]

HAN_ID,HARM_ID,Appln_id,Publn_auth,Patent_number
<int>,<int>,<int>,<chr>,<chr>
60513,60513,498018567,WO,WO2019030020
60513,60513,504430645,EP,EP3667266
60513,60513,467727684,US,US10049590
60513,60513,470567845,US,US10399692
60513,60513,417437137,EP,EP2978912


In [51]:
han_patents %>%
    .[HAN_ID %in% airbus_han_ids] %>%
    .[, .N, by=Publn_auth] %>%
    transpose() %>%
    kable('pipe', align='ccc')



| V1  | V2  | V3 |
|:---:|:---:|:--:|
| EP  | US  | WO |
| 716 | 415 | 88 |